In [2]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.0-bin-hadoop3.2')
DATA_PATH = '../data/Spark_for_Machine_Learning/Natural_Language_Processing/'

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/30 20:46:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data = spark.read.csv(DATA_PATH + 'smsspamcollection/SMSSpamCollection', 
               inferSchema=True,
               sep='\t')

## Feature engineering

In [5]:
data = (data
        .withColumnRenamed('_c0', 'class')
        .withColumnRenamed('_c1', 'sentence')
       )

In [6]:
data.groupBy('class').mean().show()

+-----+
|class|
+-----+
|  ham|
| spam|
+-----+



In [7]:
from pyspark.sql.functions import length, col
from pyspark.ml.feature import (Tokenizer, StopWordsRemover, 
                                CountVectorizer, IDF, 
                                StringIndexer, VectorAssembler)
from pyspark.ml.classification import RandomForestClassifier, LinearSVC
from pyspark.ml import Pipeline

In [8]:
data = data.withColumn('length', length(col('sentence')))
tokenizer = Tokenizer(inputCol='sentence',outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='tokens')
count_vec = CountVectorizer(inputCol='tokens',outputCol='c_vec')
idf = IDF(inputCol='c_vec',outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')
vec_assembler = VectorAssembler(inputCols=['tf_idf','length'], outputCol='features')
svm = LinearSVC()

In [9]:
pipe = Pipeline(stages=[
    tokenizer, 
    stop_remove, 
    count_vec, 
    idf, 
    ham_spam_to_numeric, 
    vec_assembler,
    svm,
])

In [ ]:
train, test = df.randomSplit([0.7, 0.3])

In [12]:
# Stratify and split
train0, test0 = data.filter(col('class')=='ham').randomSplit([0.7,0.3])
train1, test1 = data.filter(col('class')=='spam').randomSplit([0.7,0.3])

train = train0.union(train1) 
test = test0.union(test1)

In [13]:
fit_model = pipe.fit(train)

21/12/30 20:48:35 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
21/12/30 20:48:35 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
21/12/30 20:48:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
21/12/30 20:48:35 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [14]:
test_result = fit_model.transform(test)

In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [18]:
evaluator = MulticlassClassificationEvaluator()

In [23]:
evaluator.getMetricName()

'f1'

In [19]:
evaluator.evaluate(test_result)

21/12/30 20:49:45 WARN DAGScheduler: Broadcasting large task binary with size 1063.0 KiB


0.982029673473938